In [ ]:
import ipywidgets as w
import nglview as nv
import glob
import os
from gmx import GMX
import re
import time

In [ ]:
pdbview = nv.NGLWidget()
pdbview_component = None
cartoon = w.Checkbox(description='Cartoon',value=True)
licorice = w.Checkbox(description='Licorice',value=True)

def on_checkbox_click(e):
    set_pdbview_representation()

cartoon.observe(on_checkbox_click,names='value')
licorice.observe(on_checkbox_click,names='value')

In [ ]:
pdbs = w.Dropdown(
    options=glob.glob('*.pdb') + ['none'],
    value='none',
    description='Current filename',
    disabled=False,
)

def set_pdbview_representation():
    pdbview.clear()
    if cartoon.value:
        pdbview.add_cartoon(color='restype')
    if licorice.value:
        pdbview.add_licorice()


def pdb_choose(e):
    global pdbview_component
#    print('pdb_choose:',e)
    if pdbview.n_components > 0:
        pdbview.remove_component(pdbview_component)
    pdbs.value = e['new']
    if pdbs.value != 'none':
        pdbview_component = pdbview.add_component(pdbs.value)
        set_pdbview_representation()
    

pdbs.observe(pdb_choose,'value')


In [ ]:
upload = w.FileUpload(accept='pdb',multiple=False)

def on_upload_pdb(c):
#    global pdbview_component
#    print(upload.value)
    fname = list(upload.value.keys())[0]
    with open(fname,'wb') as f:
        f.write(list(upload.value.values())[0]['content'])
    
    pdbs.options = glob.glob('*.pdb') + ['none']
#    pdbs.value = fname
    upload.value.clear()
    pdb_choose({'new':fname})

#    if pdbview.n_components > 0:
#        pdbview.remove_component(pdbview_component)
#    pdbview_component = pdbview.add_component(name.value)
#    set_pdbview_representation()
    

    
    
upload.observe(on_upload_pdb,'_counter')


In [ ]:
start = w.Button(
    description='Run',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)

In [ ]:
prep = w.IntProgress(
    value=0,min=0,max=6,
    description='Prepare:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'blue'},
    orientation='horizontal'
)

minim = w.FloatProgress(
    value=0.,min=0.,max=1.,
    description='Minimize:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'yellow'},
    orientation='horizontal'
)

nvt = w.IntProgress(
    value=0,min=0,max=5,
    description='NVT:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'yellow'},
    orientation='horizontal'
)

npt = w.IntProgress(
    value=0,min=0,max=5,
    description='NPT:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'yellow'},
    orientation='horizontal'
)


err = w.Textarea(description='Error:',layout=w.Layout(width='90%',visibility='hidden'))

In [ ]:
prepare = w.VBox([w.HBox([upload, pdbs]), w.HBox([pdbview, w.VBox([cartoon, licorice])]),
               w.HBox([prep,minim, nvt,npt]), err, start])


In [ ]:
bias = w.Label('TODO')
mdrun = w.Label('TODO')

In [ ]:
mnt=os.popen('mount | grep /home/jovyan').read()
pvcid=re.search('pvc-[0-9a-z-]+',mnt).group(0)
pvc=os.popen(f'kubectl get pvc | grep {pvcid} | cut -f1 -d" "').read().rstrip()
pvc

In [ ]:
gmx = GMX(pvc=pvc,workdir='')

In [ ]:
def check_stat():
    global gmx, prep, err
    while not gmx.status().succeeded and not gmx.status().failed:
        time.sleep(1)
    if gmx.status().succeeded:
        prep.value += 1
        ret = True
    else:
        err.value = gmx.log()
        err.layout.visibility = 'visible'
        ret = False
    gmx.delete()
    return ret
        
prep.value = 0
err.value = ''
mdbox=2.0

def run_prepare(e):
    pdb = pdbs.value
    base = pdb.replace('.pdb','')
    prep.value = 1
    prep.style.bar_color = 'blue'
    err.layout.visibility = 'hidden'
    err.value = ''
    ok = True
    for cmd in [
        f"pdb2gmx -f {pdb} -o {base}.gro -p {base}.top -water tip3p -ff amber94 -ignh",
        f"editconf -f {base}.gro -o {base}-box.gro -c -d {mdbox} -bt dodecahedron",
        f"solvate -cp {base}-box.gro -cs spc216.gro -o {base}-solv.gro -p {base}.top",
        f"grompp -f ions.mdp -c {base}-solv.gro -p {base}.top -o ions.tpr",
        (f"genion -s ions.tpr -o {base}-ions.gro -p {base}.top -pname NA -nname CL -neutral","13"),
     ]:
        if isinstance(cmd,str):
            gmx.start(cmd)
        else:
            gmx.start(cmd[0],input=cmd[1])
        ok = check_stat()
        if not ok:
            break

    prep.style.bar_color = 'green' if ok else 'red'
    
start.on_click(run_prepare)

In [ ]:
main = w.Tab()
main.children = [prepare, bias, mdrun]
main.set_title(0,'Prepare molecule')
main.set_title(1,'Bias potential')
main.set_title(2,'Run MD')
display(main)

In [ ]:
"""
gmx.start(f"grompp -f minim-sol.mdp -c {base}-ions.gro -p {base}.top -o em.tpr") # XXX: error
gmx.delete()
gmx.start(f"mdrun -v -deffnm em -pin on",cores=4,gpus=1)

maxs = 500
while not gmx.status().succeeded:
    with open('em.log') as log:
        for i,l in reversed(enumerate(log)):
            if re.match('\s+Step\s+Time',l):
                s = float(re.match('\s+([[:digit:]]+)\s+',prev).group(1))
                break
            prev = l
    if s >= maxs:
        maxs += 500
    minim.value = s / maxs
    sleep(1)
"""


In [ ]:
#gmx.start(f"mdrun -v -deffnm em -pin on",cores=4,gpus=1)

In [ ]:
# print(gmx.log())

In [ ]:
!ls

In [ ]:
!date

In [ ]:
!ls /opt/gmx/lib/python3.10/site-packages/gmx/